## Merge toute les données intérim (variables des 4 piliers)

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.options.display.float_format = '{:,.3f}'.format

external_data = Path('../data/external/')
processed_data = Path('../data/processed/')
raw_data = Path('../data/raw/')
interim_data = Path('../data/interim/')

In [2]:
!ls -lh ../data/interim/

total 227M
-rw-rw-r-- 1 outini outini 822K déc.  15 23:48 acces_point_phys_service_publique.csv
-rw-rw-r-- 1 outini outini 109M déc.  15 23:54 add_geom_data_to_merged_data.trc.pqt
-rw-rw-r-- 1 outini outini 2,8K déc.  15 23:48 couv_mobile_domtom_temp.csv
-rw-rw-r-- 1 outini outini 4,0M déc.  15 23:48 data_scol_menage_demandeurEmplois.csv
-rw-rw-r-- 1 outini outini 109M déc.  15 23:54 iris_df.trc.pqt
drwxrwxr-x 1 outini outini  118 déc.  16 00:12 old
-rw-rw-r-- 1 outini outini 892K déc.  15 23:48 taux_couverture_mobile.csv
-rw-rw-r-- 1 outini outini 1,2M déc.  16 00:12 taux_couverture_THD.csv
-rw-rw-r-- 1 outini outini 706K déc.  15 23:48 taux_demandeurs_emploi.csv
-rw-rw-r-- 1 outini outini 781K déc.  15 23:48 taux_pauvrete.csv
-rw-rw-r-- 1 outini outini 999K déc.  15 23:48 variable_minimas_V2.csv


In [3]:
df1 = pd.read_csv(interim_data/'acces_point_phys_service_publique.csv')
df2 = pd.read_csv(interim_data/'data_scol_menage_demandeurEmplois.csv')
df3 = pd.read_csv(interim_data/'taux_couverture_mobile.csv')
df4 = pd.read_csv(interim_data/'taux_couverture_THD.csv')
df5 = pd.read_csv(interim_data/'taux_demandeurs_emploi.csv')    # get TX_DEMANDEUR_EMPLOIS from this one
df6 = pd.read_csv(interim_data/'taux_pauvrete.csv')
df7 = pd.read_csv(interim_data/'variable_minimas_V2.csv')

communes = pd.read_csv(raw_data/'table_insee_libcom_dep.csv').drop(columns=['Unnamed: 0'])

### Check pour avoir meme base de 3 index : (CODE_INSEE, LIBCOM, DEP) et meme taille de dataset

In [4]:
for data in [df1, df2, df3, df4, df5, df6, df7]:
    print(data.columns)
    print(data.shape)
    print()

Index(['CODE_INSEE', 'LIBCOM', 'DEP', 'ACCES_SERVICE_PUBLIC'], dtype='object')
(35013, 4)

Index(['Unnamed: 0', 'CODE_INSEE', 'LIBCOM', 'DEP', 'TX_NSCOL15P', 'TX_MENSEUL', 'TX_FAMMONO', 'TX_65ETPLUS', 'TX_25ETMOINS', 'TX_DEMANDEURS_EMPLOI'], dtype='object')
(36101, 10)

Index(['CODE_INSEE', 'LIBCOM', 'DEP', 'COUVERTURE_MOBILE'], dtype='object')
(35010, 4)

Index(['CODE_INSEE', 'DEP', 'LIBCOM', 'TAUX_COUVERTURE_THD'], dtype='object')
(35359, 4)

Index(['CODE_INSEE', 'LIBCOM', 'TX_DEMANDEUR_EMPLOIS'], dtype='object')
(35010, 3)

Index(['CODE_INSEE', 'LIBCOM', 'TX_POVERTY'], dtype='object')
(35010, 3)

Index(['Unnamed: 0', 'CODE_INSEE', 'LIBCOM', 'DEP', 'TX_RSA'], dtype='object')
(35010, 5)



### Check nouvelle taille de chaque dataset

In [5]:
indexes = ['CODE_INSEE', 'LIBCOM', 'DEP']
how = 'left'

df1 = communes.merge(df1, how=how, on=indexes).drop_duplicates(subset=indexes)
df2.drop(columns=['Unnamed: 0', 'TX_DEMANDEURS_EMPLOI'], inplace=True)
df2 = communes.merge(df2, how=how, on=indexes).drop_duplicates(subset=indexes)
df3 = communes.merge(df3, how=how, on=indexes).drop_duplicates(subset=indexes)
df4 = communes.merge(df4, how=how, on=indexes).drop_duplicates(subset=indexes)
df5 = communes.merge(df5, how=how, on=['CODE_INSEE', 'LIBCOM']).drop_duplicates(subset=indexes)
df6 = communes.merge(df6, how=how, on=['CODE_INSEE', 'LIBCOM']).drop_duplicates(subset=indexes)
df7.drop(columns=['Unnamed: 0'], inplace=True)

for data in [df1, df2, df3, df4, df5, df6, df7]:
    print(data.columns)
    print(data.shape)
    print()

Index(['CODE_INSEE', 'LIBCOM', 'DEP', 'ACCES_SERVICE_PUBLIC'], dtype='object')
(35010, 4)

Index(['CODE_INSEE', 'LIBCOM', 'DEP', 'TX_NSCOL15P', 'TX_MENSEUL', 'TX_FAMMONO', 'TX_65ETPLUS', 'TX_25ETMOINS'], dtype='object')
(35010, 8)

Index(['CODE_INSEE', 'LIBCOM', 'DEP', 'COUVERTURE_MOBILE'], dtype='object')
(35010, 4)

Index(['CODE_INSEE', 'LIBCOM', 'DEP', 'TAUX_COUVERTURE_THD'], dtype='object')
(35010, 4)

Index(['CODE_INSEE', 'LIBCOM', 'DEP', 'TX_DEMANDEUR_EMPLOIS'], dtype='object')
(35010, 4)

Index(['CODE_INSEE', 'LIBCOM', 'DEP', 'TX_POVERTY'], dtype='object')
(35010, 4)

Index(['CODE_INSEE', 'LIBCOM', 'DEP', 'TX_RSA'], dtype='object')
(35010, 4)



### MERGE

In [6]:
df = df1.merge(df2, how='inner', on=['CODE_INSEE', 'LIBCOM', 'DEP'])
df = df.merge(df3, how='inner', on=['CODE_INSEE', 'LIBCOM', 'DEP'])
df = df.merge(df4, how='inner', on=['CODE_INSEE', 'LIBCOM', 'DEP'])
df = df.merge(df5, how='inner', on=['CODE_INSEE', 'LIBCOM', 'DEP'])
df = df.merge(df6, how='inner', on=['CODE_INSEE', 'LIBCOM', 'DEP'])
df = df.merge(df7, how='inner', on=['CODE_INSEE', 'LIBCOM', 'DEP'])

print(df.shape)

(35010, 14)


In [7]:
df.head()

,CODE_INSEE,LIBCOM,DEP,ACCES_SERVICE_PUBLIC,TX_NSCOL15P,TX_MENSEUL,TX_FAMMONO,TX_65ETPLUS,TX_25ETMOINS,COUVERTURE_MOBILE,TAUX_COUVERTURE_THD,TX_DEMANDEUR_EMPLOIS,TX_POVERTY,TX_RSA
0,01001,L'Abergement-Clémenciat,01,0,52.154,22.824,4.832,23.779,11.309,0.000,98.417,nan,nan,nan
1,01002,L'Abergement-de-Varey,01,0,40.426,28.571,4.762,23.404,7.979,0.500,37.173,nan,nan,nan
2,01004,Ambérieu-en-Bugey,01,0,49.475,41.515,10.216,20.734,16.620,1.000,38.403,21.755,26.171,2.594
3,01005,Ambérieux-en-Dombes,01,1,53.575,21.076,6.602,19.488,13.854,1.000,34.190,nan,nan,nan
4,01006,Ambléon,01,0,53.061,36.364,9.091,26.531,10.204,0.800,0.000,nan,nan,nan


In [8]:
#for col in 'ACCES_SERVICE_PUBLIC COUVERTURE_MOBILE TAUX_COUVERTURE_THD'.split():
#    df[col] = 1 - df[col]

In [9]:
df

,CODE_INSEE,LIBCOM,DEP,ACCES_SERVICE_PUBLIC,TX_NSCOL15P,TX_MENSEUL,TX_FAMMONO,TX_65ETPLUS,TX_25ETMOINS,COUVERTURE_MOBILE,TAUX_COUVERTURE_THD,TX_DEMANDEUR_EMPLOIS,TX_POVERTY,TX_RSA
0,01001,L'Abergement-Clémenciat,01,0,52.154,22.824,4.832,23.779,11.309,0.000,98.417,nan,nan,nan
1,01002,L'Abergement-de-Varey,01,0,40.426,28.571,4.762,23.404,7.979,0.500,37.173,nan,nan,nan
2,01004,Ambérieu-en-Bugey,01,0,49.475,41.515,10.216,20.734,16.620,1.000,38.403,21.755,26.171,2.594
3,01005,Ambérieux-en-Dombes,01,1,53.575,21.076,6.602,19.488,13.854,1.000,34.190,nan,nan,nan
4,01006,Ambléon,01,0,53.061,36.364,9.091,26.531,10.204,0.800,0.000,nan,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35005,97613,M'Tsangamouji,976,0,nan,nan,nan,nan,nan,1.000,76.202,nan,nan,nan
35006,97614,Ouangani,976,0,nan,nan,nan,nan,nan,1.000,75.919,nan,nan,nan
35007,97615,Pamandzi,976,1,nan,nan,nan,nan,nan,1.000,75.365,nan,51.412,nan
35008,97616,Sada,976,1,nan,nan,nan,nan,nan,1.000,66.325,nan,49.199,nan


### Cela résulte en 11 colonnes pour calculer le score à savoir: 

In [10]:
list(df.drop(columns=indexes).columns)

['ACCES_SERVICE_PUBLIC',
 'TX_NSCOL15P',
 'TX_MENSEUL',
 'TX_FAMMONO',
 'TX_65ETPLUS',
 'TX_25ETMOINS',
 'COUVERTURE_MOBILE',
 'TAUX_COUVERTURE_THD',
 'TX_DEMANDEUR_EMPLOIS',
 'TX_POVERTY',
 'TX_RSA']

### + 3 colonnes d'index :

In [11]:
list(indexes)

['CODE_INSEE', 'LIBCOM', 'DEP']

## Indiquer si une variable est NaN par ligne + Mapping *EPCI*

In [12]:
boolean_df = df.isna().drop(["CODE_INSEE", "LIBCOM", "DEP"], axis=1)
# on renomme les colonnes pour simplifier la jointure
for col in boolean_df:
    boolean_df= boolean_df.rename(columns={col: str(col)+"_isNaN"})

# Merge du dataset + indication isNaN par variable
df_with_NaN = pd.merge(df, boolean_df, left_index=True, right_index=True)
df_with_NaN

,CODE_INSEE,LIBCOM,DEP,ACCES_SERVICE_PUBLIC,TX_NSCOL15P,TX_MENSEUL,TX_FAMMONO,TX_65ETPLUS,TX_25ETMOINS,COUVERTURE_MOBILE,TAUX_COUVERTURE_THD,TX_DEMANDEUR_EMPLOIS,TX_POVERTY,TX_RSA,ACCES_SERVICE_PUBLIC_isNaN,TX_NSCOL15P_isNaN,TX_MENSEUL_isNaN,TX_FAMMONO_isNaN,TX_65ETPLUS_isNaN,TX_25ETMOINS_isNaN,COUVERTURE_MOBILE_isNaN,TAUX_COUVERTURE_THD_isNaN,TX_DEMANDEUR_EMPLOIS_isNaN,TX_POVERTY_isNaN,TX_RSA_isNaN
0,01001,L'Abergement-Clémenciat,01,0,52.154,22.824,4.832,23.779,11.309,0.000,98.417,nan,nan,nan,False,False,False,False,False,False,False,False,True,True,True
1,01002,L'Abergement-de-Varey,01,0,40.426,28.571,4.762,23.404,7.979,0.500,37.173,nan,nan,nan,False,False,False,False,False,False,False,False,True,True,True
2,01004,Ambérieu-en-Bugey,01,0,49.475,41.515,10.216,20.734,16.620,1.000,38.403,21.755,26.171,2.594,False,False,False,False,False,False,False,False,False,False,False
3,01005,Ambérieux-en-Dombes,01,1,53.575,21.076,6.602,19.488,13.854,1.000,34.190,nan,nan,nan,False,False,False,False,False,False,False,False,True,True,True
4,01006,Ambléon,01,0,53.061,36.364,9.091,26.531,10.204,0.800,0.000,nan,nan,nan,False,False,False,False,False,False,False,False,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35005,97613,M'Tsangamouji,976,0,nan,nan,nan,nan,nan,1.000,76.202,nan,nan,nan,False,True,True,True,True,True,False,False,True,True,True
35006,97614,Ouangani,976,0,nan,nan,nan,nan,nan,1.000,75.919,nan,nan,nan,False,True,True,True,True,True,False,False,True,True,True
35007,97615,Pamandzi,976,1,nan,nan,nan,nan,nan,1.000,75.365,nan,51.412,nan,False,True,True,True,True,True,False,False,True,False,True
35008,97616,Sada,976,1,nan,nan,nan,nan,nan,1.000,66.325,nan,49.199,nan,False,True,True,True,True,True,False,False,True,False,True


In [13]:
# EPCI
epci_xls = pd.ExcelFile(raw_data/ "Intercommunalité - Métropole au 01-01-2020.xlsx")
df_epci = pd.read_excel(epci_xls, 'Composition_communale', skiprows=5)
df_epci

,CODGEO,LIBGEO,EPCI,LIBEPCI,DEP,REG
0,01001,L'Abergement-Clémenciat,200069193,CC de la Dombes,01,84
1,01002,L'Abergement-de-Varey,240100883,CC de la Plaine de l'Ain,01,84
2,01004,Ambérieu-en-Bugey,240100883,CC de la Plaine de l'Ain,01,84
3,01005,Ambérieux-en-Dombes,200042497,CC Dombes Saône Vallée,01,84
4,01006,Ambléon,200040350,CC Bugey Sud,01,84
...,...,...,...,...,...,...
34963,97613,M'Tsangamouji,200059871,CC du Centre-Ouest,976,6
34964,97614,Ouangani,200059871,CC du Centre-Ouest,976,6
34965,97615,Pamandzi,200050532,CC de Petite-Terre,976,6
34966,97616,Sada,200059871,CC du Centre-Ouest,976,6


In [14]:
df_epci = df_epci[["CODGEO", "EPCI", "LIBEPCI"]].rename(columns={"CODGEO": "CODE_INSEE"})

# MERGE 
df = pd.merge(df_epci, df_with_NaN, on="CODE_INSEE", how='right').sort_values('CODE_INSEE')

In [15]:
df

,CODE_INSEE,EPCI,LIBEPCI,LIBCOM,DEP,ACCES_SERVICE_PUBLIC,TX_NSCOL15P,TX_MENSEUL,TX_FAMMONO,TX_65ETPLUS,TX_25ETMOINS,COUVERTURE_MOBILE,TAUX_COUVERTURE_THD,TX_DEMANDEUR_EMPLOIS,TX_POVERTY,TX_RSA,ACCES_SERVICE_PUBLIC_isNaN,TX_NSCOL15P_isNaN,TX_MENSEUL_isNaN,TX_FAMMONO_isNaN,TX_65ETPLUS_isNaN,TX_25ETMOINS_isNaN,COUVERTURE_MOBILE_isNaN,TAUX_COUVERTURE_THD_isNaN,TX_DEMANDEUR_EMPLOIS_isNaN,TX_POVERTY_isNaN,TX_RSA_isNaN
0,01001,200069193,CC de la Dombes,L'Abergement-Clémenciat,01,0,52.154,22.824,4.832,23.779,11.309,0.000,98.417,nan,nan,nan,False,False,False,False,False,False,False,False,True,True,True
1,01002,240100883,CC de la Plaine de l'Ain,L'Abergement-de-Varey,01,0,40.426,28.571,4.762,23.404,7.979,0.500,37.173,nan,nan,nan,False,False,False,False,False,False,False,False,True,True,True
2,01004,240100883,CC de la Plaine de l'Ain,Ambérieu-en-Bugey,01,0,49.475,41.515,10.216,20.734,16.620,1.000,38.403,21.755,26.171,2.594,False,False,False,False,False,False,False,False,False,False,False
3,01005,200042497,CC Dombes Saône Vallée,Ambérieux-en-Dombes,01,1,53.575,21.076,6.602,19.488,13.854,1.000,34.190,nan,nan,nan,False,False,False,False,False,False,False,False,True,True,True
4,01006,200040350,CC Bugey Sud,Ambléon,01,0,53.061,36.364,9.091,26.531,10.204,0.800,0.000,nan,nan,nan,False,False,False,False,False,False,False,False,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34960,97613,200059871,CC du Centre-Ouest,M'Tsangamouji,976,0,nan,nan,nan,nan,nan,1.000,76.202,nan,nan,nan,False,True,True,True,True,True,False,False,True,True,True
34961,97614,200059871,CC du Centre-Ouest,Ouangani,976,0,nan,nan,nan,nan,nan,1.000,75.919,nan,nan,nan,False,True,True,True,True,True,False,False,True,True,True
34962,97615,200050532,CC de Petite-Terre,Pamandzi,976,1,nan,nan,nan,nan,nan,1.000,75.365,nan,51.412,nan,False,True,True,True,True,True,False,False,True,False,True
34963,97616,200059871,CC du Centre-Ouest,Sada,976,1,nan,nan,nan,nan,nan,1.000,66.325,nan,49.199,nan,False,True,True,True,True,True,False,False,True,False,True


In [16]:
df.to_csv(processed_data/'MERGE_data.csv', index=False)

### Check missing values (NaN --> median par département)

In [17]:
# Pourcentage des valeurs manquantes par colonne
df_na = df.isna().sum().sort_values(ascending=False)
df_na = df_na / len(df) * 100
df_na

TX_RSA                       97.181
TX_DEMANDEUR_EMPLOIS         96.638
TX_POVERTY                   83.973
TAUX_COUVERTURE_THD           0.494
EPCI                          0.129
LIBEPCI                       0.129
TX_MENSEUL                    0.089
TX_FAMMONO                    0.089
TX_NSCOL15P                   0.069
TX_65ETPLUS                   0.069
TX_25ETMOINS                  0.069
COUVERTURE_MOBILE             0.000
LIBCOM                        0.000
DEP                           0.000
ACCES_SERVICE_PUBLIC          0.000
TX_RSA_isNaN                  0.000
TX_POVERTY_isNaN              0.000
ACCES_SERVICE_PUBLIC_isNaN    0.000
TX_NSCOL15P_isNaN             0.000
TX_MENSEUL_isNaN              0.000
TX_FAMMONO_isNaN              0.000
TX_65ETPLUS_isNaN             0.000
TX_25ETMOINS_isNaN            0.000
COUVERTURE_MOBILE_isNaN       0.000
TAUX_COUVERTURE_THD_isNaN     0.000
TX_DEMANDEUR_EMPLOIS_isNaN    0.000
CODE_INSEE                    0.000
dtype: float64

### Remplir les NaN par médiane par dép.

In [18]:
for col in df.drop(columns=indexes + list(df.columns[-11:].values) + list(['EPCI','LIBEPCI'])).columns:
    df[col] = df[col].fillna(df.groupby('DEP')[col].transform('median'))

#df['TX_POVERTY'] = df['TX_POVERTY'].fillna(df.groupby('DEP')['TX_POVERTY'].agg('median'))

In [19]:
# Pourcentage des valeurs manquantes par colonne
df_na = df.isna().sum().sort_values(ascending=False)
df_na = df_na / len(df) * 100
df_na

TX_DEMANDEUR_EMPLOIS         0.300
TX_RSA                       0.300
EPCI                         0.129
LIBEPCI                      0.129
TAUX_COUVERTURE_THD          0.057
TX_NSCOL15P                  0.049
TX_MENSEUL                   0.049
TX_FAMMONO                   0.049
TX_65ETPLUS                  0.049
TX_25ETMOINS                 0.049
LIBCOM                       0.000
DEP                          0.000
ACCES_SERVICE_PUBLIC         0.000
COUVERTURE_MOBILE            0.000
TX_RSA_isNaN                 0.000
TX_POVERTY_isNaN             0.000
TX_POVERTY                   0.000
ACCES_SERVICE_PUBLIC_isNaN   0.000
TX_NSCOL15P_isNaN            0.000
TX_MENSEUL_isNaN             0.000
TX_FAMMONO_isNaN             0.000
TX_65ETPLUS_isNaN            0.000
TX_25ETMOINS_isNaN           0.000
COUVERTURE_MOBILE_isNaN      0.000
TAUX_COUVERTURE_THD_isNaN    0.000
TX_DEMANDEUR_EMPLOIS_isNaN   0.000
CODE_INSEE                   0.000
dtype: float64

#### Il reste quelques données manquantes au niveau de DOMTOM sur la couverture mobile par ex. (en cours ...)

In [20]:
df.to_csv(processed_data/'MERGE_data_clean.csv', index=False)

In [21]:
df

,CODE_INSEE,EPCI,LIBEPCI,LIBCOM,DEP,ACCES_SERVICE_PUBLIC,TX_NSCOL15P,TX_MENSEUL,TX_FAMMONO,TX_65ETPLUS,TX_25ETMOINS,COUVERTURE_MOBILE,TAUX_COUVERTURE_THD,TX_DEMANDEUR_EMPLOIS,TX_POVERTY,TX_RSA,ACCES_SERVICE_PUBLIC_isNaN,TX_NSCOL15P_isNaN,TX_MENSEUL_isNaN,TX_FAMMONO_isNaN,TX_65ETPLUS_isNaN,TX_25ETMOINS_isNaN,COUVERTURE_MOBILE_isNaN,TAUX_COUVERTURE_THD_isNaN,TX_DEMANDEUR_EMPLOIS_isNaN,TX_POVERTY_isNaN,TX_RSA_isNaN
0,01001,200069193,CC de la Dombes,L'Abergement-Clémenciat,01,0,52.154,22.824,4.832,23.779,11.309,0.000,98.417,18.910,19.559,2.594,False,False,False,False,False,False,False,False,True,True,True
1,01002,240100883,CC de la Plaine de l'Ain,L'Abergement-de-Varey,01,0,40.426,28.571,4.762,23.404,7.979,0.500,37.173,18.910,19.559,2.594,False,False,False,False,False,False,False,False,True,True,True
2,01004,240100883,CC de la Plaine de l'Ain,Ambérieu-en-Bugey,01,0,49.475,41.515,10.216,20.734,16.620,1.000,38.403,21.755,26.171,2.594,False,False,False,False,False,False,False,False,False,False,False
3,01005,200042497,CC Dombes Saône Vallée,Ambérieux-en-Dombes,01,1,53.575,21.076,6.602,19.488,13.854,1.000,34.190,18.910,19.559,2.594,False,False,False,False,False,False,False,False,True,True,True
4,01006,200040350,CC Bugey Sud,Ambléon,01,0,53.061,36.364,9.091,26.531,10.204,0.800,0.000,18.910,19.559,2.594,False,False,False,False,False,False,False,False,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34960,97613,200059871,CC du Centre-Ouest,M'Tsangamouji,976,0,nan,nan,nan,nan,nan,1.000,76.202,nan,64.840,nan,False,True,True,True,True,True,False,False,True,True,True
34961,97614,200059871,CC du Centre-Ouest,Ouangani,976,0,nan,nan,nan,nan,nan,1.000,75.919,nan,64.840,nan,False,True,True,True,True,True,False,False,True,True,True
34962,97615,200050532,CC de Petite-Terre,Pamandzi,976,1,nan,nan,nan,nan,nan,1.000,75.365,nan,51.412,nan,False,True,True,True,True,True,False,False,True,False,True
34963,97616,200059871,CC du Centre-Ouest,Sada,976,1,nan,nan,nan,nan,nan,1.000,66.325,nan,49.199,nan,False,True,True,True,True,True,False,False,True,False,True
